# Анализ данных

In [ ]:
import sys
import os
import pydicom
from pydicom import dcmread
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
from typing import Union, Tuple, List, Dict

from utils.data_analysis import visualize_dicom 
from utils.data_analysis import apply_ct_window
from utils.data_analysis import print_dicom_rescale_info
from utils.data_analysis import get_number_of_frames
from utils.data_analysis import analyze_dicom_volume
from utils.metadata_extraction import analyze_dicom_series
from utils.metadata_extraction import visualize_dicom_geometry

# Тестовые данные

## Норма

In [ ]:
ds = dcmread('../data/raw/norma_anon/norma_anon/10000E6B_anon', force=True)

# Выводим информацию
print("DICOM информация:")
print(f"Модальность: {ds.Modality}")
print(f"Размер: {ds.Rows}x{ds.Columns}")
print(f"Срезы: {getattr(ds, 'NumberOfFrames', 1)}")

# Показываем изображение
pixel_data = ds.pixel_array
num_frames = getattr(ds, 'NumberOfFrames', 1)

if num_frames > 1:
    plt.imshow(pixel_data[num_frames//2])
else:
    plt.imshow(pixel_data)

plt.axis('off')
plt.show()

In [ ]:
# Путь к папке с DICOM файлами
folder_path = '../data/raw/norma_anon/norma_anon'
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

In [ ]:
visualize_dicom(
        folder_path,
        step=29,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

Посчитаем статистику 

**mean_hu** (средняя плотность тканей в единицах Хаунсфилда (Hounsfield Units/HU))

Показывает общую "среднюю яркость" исследования

**std_hu** (Стандартное отклонение HU):

Показывает, насколько разнородна плотность тканей в исследовании

Высокое значение означает большую вариабельность плотностей (например, при наличии и воздушных, и плотных участков)

Низкое значение означает однородность тканей

**pct_low_attenuation** (Процент низкоаттенуированной ткани):

Показывает долю ткани с низкой плотностью (ниже определенного порога, обычно -950 HU)

Важный показатель при анализе легких, особенно для оценки эмфиземы

Чем выше этот процент, тем больше участков с патологически низкой плотностью

In [ ]:
# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

### Метаданные

In [ ]:
# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

In [ ]:
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

###  Анализ параметров сканирования КТ норма

**Обнаружено:** Реконструкция с перекрытием (overlapping reconstruction) — стандартная практика для улучшения качества 3D-визуализации.

---

** Расчеты и обоснование:**

**Исходные данные:**
- Количество срезов: 451
- Z-диапазон: 1954.50 - 1594.50 = 360.00 мм
- Slice Thickness: 1.0 мм (толщина реконструированного среза)
- Spiral Pitch Factor: 1.388

**Расчет шага реконструкции:**
Δz = Общий диапазон / (N срезов - 1) = 360.00 / 450 = 0.80 мм

**Расчет коэффициента перекрытия:**
Overlap = 1 - (Шаг / Толщина) = 1 - (0.8 / 1.0) = 0.2 (20%)


**📊 Технические параметры:**
- **Data Collection Diameter:** 500.00 мм (физический сбор данных)
- **Reconstruction Diameter:** 400.39 мм (поле реконструкции)
- **Покрытие за оборот:** Pitch × Коллимация = 1.388 × (N×1.0 мм)

**✅ Вывод:** Система работает в штатном режиме. Перекрытие в 20% улучшает качество 3D-реконструкций и повышает точность диагностики за счет исключения "ступенчатости" и улучшения видимости мелких образований на нескольких соседних срезах.
Таким образом реальный размер вокселя: 0.782×0.782×0.8 мм

## Пневмония

In [ ]:
# Путь к папке с DICOM файлами
folder_path = '../data/raw/pneumonia_anon/pneumonia_anon'
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

In [ ]:
visualize_dicom(
        folder_path,
        step=23,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

In [ ]:
# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

In [ ]:
# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
#ds

In [ ]:
pnevma_metadata = analyze_dicom_series(folder_path)
pnevma_metadata

In [ ]:
visualize_dicom_geometry(pnevma_metadata)

## Пневмоторекс

In [ ]:
# Путь к папке с DICOM файлами
folder_path = '../data/raw/pneumotorax_anon/pneumotorax_anon'
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

In [ ]:
file_path = "../data/raw/pneumotorax_anon/pneumotorax_anon/10000CF6_anon"
frames = get_number_of_frames(file_path)
print(f"Файл содержит {frames} фреймов.")

In [ ]:
print_dicom_rescale_info(folder_path)

In [ ]:
visualize_dicom(
        file_path,
        step=29,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True,
        start_frame=1,
        max_frames=464
    )

In [ ]:
# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    file_path, 
    low_att_threshold=-950,
    visualize=True
)

In [ ]:
# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

In [ ]:
ds

In [ ]:
pnevmotorex_metadata = analyze_dicom_series(folder_path)
pnevmotorex_metadata

## Сравнение параметров КТ-снимков: Норма vs Пневмония vs Пневмоторакс

| Параметр                  | Случай 1 (Норма)      | Случай 2 (Пневмония)   | Случай 3 (Пневмоторакс) | Комментарий |
|---------------------------|------------------------|-------------------------|--------------------------|-------------|
| **Производитель сканера** | TOSHIBA                | Siemens                 | TOSHIBA Aquilion         | разные алгоритмы реконструкции и шумовые характеристики |
| **Напряжение (кВ)**       | 120                    | 100                     | 120                      | влияет на контрастность тканей и проникающую способность излучения; **требуется восстановление или оценка по протоколу** |
| **Ток (мА) / Экспозиция** | 80 мА / 40 мАс         | 673 мА / 148 мАс        | 116/58                     | сильно влияет на уровень шума и качество изображения; **данные отсутствуют — может потребоваться аппроксимация** |
| **Доза (мГр)**            | 2.6                    | 8.07                    | 27.5                      | прямо влияет на соотношение сигнал/шум (SNR); **необходимо дополнить из протокола, если доступно** |
| **Размер пикселя (мм)**   | 0.782                  | 0.752                   | 0.782      | **требуется ресемплинг к единому разрешению для обучения модели**; предположительно ~1.0 мм/пиксель |
| **Интерцепт пересчёта**   | 0                      | -8192                   | 0                        | **определяет абсолютные значения HU; без коррекции модель увидит разные "единицы измерения"** |
| **Представление пикселей**| Со знаком              | Без знака               | Со знаком                | **влияет на диапазон значений и интерпретацию HU; требует нормализации** |



# Подбор окон

Шкала Хаунсфилда (HU) — это количественная шкала рентгеновской плотности, используемая в компьютерной томографии. За основу приняты:
- **Вода = 0 HU**
- **Воздух = -1000 HU**

**Основные ткани и материалы**

| Ткань / Материал         | HU Значения              |
|--------------------------|--------------------------|
| Воздух                   | -1000 HU                 |
| Жир                      | -120 to -90 HU           |
| Вода                     | 0 HU                     |
| Мягкие ткани             | 20–50 HU                 |
| Кровь                    | 30–45 HU                 |
| Мышцы                    | 35–55 HU                 |
| Печень                   | 40–60 HU                 |
| Кости                    | 300–1000+ HU             |


**Дополнительные материалы и состояния**

| Ткань / Материал         | HU Значения              |
|--------------------------|--------------------------|
| Лёгкие (инспирация)      | -800 to -600 HU          |
| Лёгкие (экспирация)      | -500 to -200 HU          |
| Контрастное вещество     | 100–3000 HU              |
| Кальцинаты               | 150–2000 HU              |
| Зубная эмаль             | 2000–3000 HU             |


**Клиническое значение HU**

- **Отрицательные значения**:  
  ➤ Воздух, жир, лёгочная ткань.

- **Около 0 HU**:  
  ➤ Вода, спинномозговая жидкость (CSF).

- **Положительные значения (умеренные)**:  
  ➤ Мягкие ткани, внутренние органы (печень, селезёнка, почки, мышцы).

- **Высокие положительные значения**:  
  ➤ Кости, кальцификаты, металлы, контрастные агенты.


In [ ]:
# Загружаем DICOM файл
file_path = '../data/raw/pneumonia_anon/pneumonia_anon/100003A0_anon'
#file_path = '../data/raw/norma_anon/norma_anon/10000E6B_anon'
ds = dcmread(file_path, force=True)

# Получаем изображение в HU (pixel_array уже в HU для CT)
ct_image = ds.pixel_array.astype(np.float32)

# Проверяем наличие Rescale Slope/Intercept в метаданных
rescale_slope = getattr(ds, 'RescaleSlope', 1.0)
rescale_intercept = getattr(ds, 'RescaleIntercept', 0.0)

# Если нужно преобразовать в HU 
if rescale_slope != 1.0 or rescale_intercept != 0.0:
    ct_image = ct_image * rescale_slope + rescale_intercept

print(f"Rescale Slope: {rescale_slope}, Intercept: {rescale_intercept}")
print(f"Диапазон HU значений: {ct_image.min():.1f} - {ct_image.max():.1f}")

optimal_windows = {
    'Легкие (детализация)': (-600, 1500),
    'Медиастинум (органы)': (40, 400),
    'Кости (ребра, позвоночник)': (400, 1500) # 
}

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for (ax, (window_name, (center, width))) in zip(axes, optimal_windows.items()):
    windowed_image = apply_ct_window(ct_image, center, width)
    ax.imshow(windowed_image)
    ax.set_title(f'{window_name}\nC:{center}, W:{width}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def interactive_ct_window(window_center=40, window_width=400):
    """Интерактивная настройка окна"""
    windowed_image = apply_ct_window(ct_image, window_center, window_width)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(windowed_image, cmap='gray')
    plt.title(f'CT Window: Center={window_center}, Width={window_width}')
    plt.axis('off')
    plt.show()
    
    # Показываем гистограмму HU значений
    plt.figure(figsize=(10, 3))
    plt.hist(ct_image.flatten(), bins=200, range=(-1000, 1000), alpha=0.7)
    plt.axvline(window_center - window_width/2, color='r', linestyle='--', label='Window Min')
    plt.axvline(window_center + window_width/2, color='r', linestyle='--', label='Window Max')
    plt.axvline(window_center, color='g', linestyle='-', label='Center')
    plt.xlabel('HU Values')
    plt.ylabel('Frequency')
    plt.legend()
    plt.title('Histogram of HU Values with Current Window')
    plt.show()


In [ ]:
# Создаем интерактивные ползунки
interact(interactive_ct_window,
         window_center=IntSlider(value=40, min=-1000, max=1000, step=10, description='Center'),
         window_width=IntSlider(value=400, min=50, max=2000, step=50, description='Width'))

 **Настройки окон для КТ:**
- **Лёгочные окна**: Центр `-600`, Ширина `1500`
- **Мягкотканые окна**: Центр `40`, Ширина `400`
- **Костные окна**: Центр `400`, Ширина `1500`

# Обучающий датасет

In [ ]:
# Базовый путь к сетевому хранилищу
base_path = "/Volumes/DS/DataSets/MosMed/ds_cancer_1/MosMedData-LDCT-LUNGCR-type I-v 1/"

# Полные пути к файлам
excel_path = os.path.join(base_path, 'dataset_registry.xlsx')
studies_dir = os.path.join(base_path, 'studies')

## Датасет 1 MosMedData НДКТ с признаками рака легкого тип I

MosMedData НДКТ с признаками рака легкого тип I (https://mosmed.ai/datasets/datasets/mm/)

Есть отклонение по оси z

In [ ]:
base_path = '../data/external/dataset_1/'

In [ ]:
target_path = os.path.join(base_path, 'dataset_registry.xlsx') 
target = pd.read_excel(target_path)

In [ ]:
pd.set_option('display.max_colwidth', None)
target.head()

In [ ]:
target.loc[target['study_instance_anon'] == '1.2.643.5.1.13.13.12.2.77.8252.05031412000213100215131209081403']

### С патологией

In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, '1.2.643.5.1.13.13.12.2.77.8252.02000015141207091410081401020409/1.2.643.5.1.13.13.12.2.77.8252.09041515140805110209100012140804') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

visualize_dicom(
        folder_path,
        step=25,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

### Без патологии

В папке с исследованием 2-е папки. В одной со срезом 1мм, в другой 5мм. Интересно оставить более детальные срезы.

In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, '1.2.643.5.1.13.13.12.2.77.8252.01131513100502150809051405021408/1.2.643.5.1.13.13.12.2.77.8252.05111114100400101509070109080513') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

In [ ]:
visualize_dicom(
        folder_path,
        step=55,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

In [ ]:
# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)



In [ ]:
# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)


In [ ]:
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
interact(interactive_ct_window,
         window_center=IntSlider(value=40, min=-1000, max=1000, step=10, description='Center'),
         window_width=IntSlider(value=400, min=50, max=2000, step=50, description='Width'))

In [ ]:
visualize_dicom_geometry(norma_metadata)

# Датасет 2 CT_LUNGCANCER_500

In [ ]:
base_path = '../data/external/dataset_2/'
target_path = os.path.join(base_path, 'dataset_registry.csv') 
target = pd.read_csv(target_path)
pd.set_option('display.max_colwidth', None)
target.head()

Несколько врачебных мнений. Большое доверие. Выделяем норму - кадры по которым все мнения врачей единогласно нет. 
Важно выделить в не норму те исследования, где мнения расходятся и врач выделяет аномалии. 

Разметка в json
Список файлов в cvs

## Без патологии

In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, 'RLAD42D007-22426_RLS6A01002SVR_2418816') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

visualize_dicom(
        folder_path,
        step=75,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

## Только патология


In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, 'RLAD42D007-23196_RLS6A01002SVR_259658') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

visualize_dicom(
        folder_path,
        step=25,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

# Датасет 3 MosMedData КТ с признаками коронавирусной инфекции (COVID-19) тип I


В некоторых папках (у некоторых пациентов) есть несколько исследований. Вторе, где меньшее число файлов - это ренген.

In [ ]:
base_path = '../data/external/dataset_3/'
target_path = os.path.join(base_path, 'dataset_registry.xlsx') 
target = pd.read_excel(target_path, index_col=None)
target = target.drop(columns='Unnamed: 0')
pd.set_option('display.max_colwidth', None)
target.head()

In [ ]:
target.loc[target['pathology'] == 0].count()

In [ ]:
target.loc[target['study_instance_anon']=='1.2.643.5.1.13.13.12.2.77.8252.11020915000408000111031313111206']

## С патологией

In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, '1.2.643.5.1.13.13.12.2.77.8252.03000205120511100413101200090913/1.2.643.5.1.13.13.12.2.77.8252.12060113080214100305091010000715') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

visualize_dicom(
        folder_path,
        step=75,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

# Датасет 4 MosMedData КТ с признаками коронавирусной инфекции (COVID-19) тип I

In [ ]:
base_path = '../data/external/dataset_4/'
target_path = os.path.join(base_path, 'dataset_registry.xlsx') 
target = pd.read_excel(target_path, index_col=None)
target = target.drop(columns='Unnamed: 0')
pd.set_option('display.max_colwidth', None)
target.head()

In [ ]:
target.loc[target['study_instance_anon']=='1.2.643.5.1.13.13.12.2.77.8252.11070002040215140414030109070413']

## С патологией

## Без патологии

In [ ]:
# Путь к папке с DICOM файлами
folder_path = os.path.join(base_path, '1.2.643.5.1.13.13.12.2.77.8252.11070002040215140414030109070413/1.2.643.5.1.13.13.12.2.77.8252.10110005120514120508130113070906') 
dicom_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
dicom_files.sort()

print(f"Найдено {len(dicom_files)} DICOM файлов")

visualize_dicom(
        folder_path,
        step=75,
        grid_rows=4,
        grid_cols=4,
        apply_rescaling=True,
        apply_windowing=True
    )

# Анализ всего объема 
mean_hu, std_hu, pct_low, metadata = analyze_dicom_volume(
    folder_path, 
    low_att_threshold=-950,
    visualize=True
)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)

# Создаем список для хранения метаданных
metadata_list = []

# Читаем метаданные из первого файла (они обычно одинаковые для всех срезов)
first_file = os.path.join(folder_path, dicom_files[0])
ds = dcmread(first_file, force=True)
norma_metadata = analyze_dicom_series(folder_path)
norma_metadata

In [ ]:
visualize_dicom_geometry(norma_metadata)

# Метаданные, для обучения

Среднее hu hu_volume = pixel_data * slope + intercept из файла data_analysis

pct_low_attenuation = (low_att_voxels / total_voxels) * 100

low_att_volume_cm3 = low_att_voxels * voxel_volume_mm3 / 1000  # в см³

In [ ]:
data = {
    'study_name': ['01', '02', '03', '04', '05', '06'],
    'target': [0, 0, 0, 1, 1, 1]
}

df = pd.DataFrame(data)

# Указываем путь для сохранения (измени на свой!)
output_path = "../data/processed/train/target.csv"  

# Сохраняем в CSV
df.to_csv(output_path, index=False)  

print(f"Датасет успешно сохранён по пути: {output_path}")

In [ ]:
ds_2 = pd.read_csv('../reports/pathology_results.csv')
ds_2.head()

In [ ]:
ds_2.loc[ds_2['patology']==0].count()